# Anwendungsfälle
In diesem Abschnitt werden einige praktische Anwendungsfälle der Textanalyse bearbeitet.

In [ ]:
from collections import defaultdict

import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from transformers import pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import pke
import gensim.corpora as corpora
from gensim.models import LdaModel

stopwords_and_punctuation = set((*stopwords.words('english'), ',', '.', ';', ':', '(', ')', '!', '?', "'", '’'))

## Inhaltsverzeichnis
- [Sentimentanalyse](#Sentimentanalyse)
- [Keyphrase- und Keyword-Extraktion](#Keyphrase--und-Keyword-Extraktion)
- [Topic Modeling](#Topic-Modeling)
- [Zusammenfassung](#Zusammenfassung)

## Sentimentanalyse
Sentimentanalyse beschreibt die Extraktion von Empfindungen und Gefühlen. Abstrakter formuliert soll einem Text eine Stimmung zugeordnet werden, welche die Intention in positiv, negativ oder neutral kategorisiert.

In [ ]:
reviews = [
    'I really like the new design of your website!',
    'I was on hold for 30 minutes. Do I have to say more?',
    'At least the meal was still hot.'
]

Ein naiver Ansatz versieht jedes (relevante) Token mit einem Wert. Dabei steht $-1$ für negativ, $0$ für neutral und $1$ für positiv. Wird für jedes Token der Wert aus dem Sentiment-Wörterbuch entnommen und für eine Aussage aufsummiert, ergibt sich zumindest eine grobe Richtung der Intention.

In [ ]:
sentiment_dict = {
    'like': 1,
    'new': 1,
    'hold': -1,
    'least': -1,
    'hot': 1
}

for statement in reviews:
    tokens = [token for token in word_tokenize(statement) if token not in stopwords_and_punctuation]
    sentiment = sum(sentiment_dict.get(token, 0) for token in tokens)

    print(f'{sentiment=:>2} {statement=}')

Diese Methode hat jedoch mehrere Nachteile. Zuerst muss natürlich das Wörterbuch erstellt werden, das zumindest teilweise von der Situation abhängig ist, in dem die Statements abgegeben werden. Außerdem ist es mit lexikonbasierten Ansätzen in der Regel nicht möglich Mehrdeutigkeiten, Negationen oder auch Ironie korrekt zu verarbeiten. Ebenso sind Wörter häufig gar nicht alleinstehend bewertbar: Eine kurze Wartezeit ist etwas Gutes, obwohl die Wartezeit selbst nichts erstrebenswertes ist. Glück ist ebenfalls etwas Gutes, aber wenn es nur kurz währte, ist die Aussage wohl eher negativ gemeint. Der Kontext ist also entscheidend.

Mit Hilfe vortrainierter Modelle, die beispielsweise auf der Transformer-Architektur basieren, lässt sich der Kontext eines Worte viel besser bewerten. Das Training wie auch die Bewertung ist aber natürlich deutlich rechenintensiver.

In [ ]:
sentiment_pipeline = pipeline('sentiment-analysis',
                              model='distilbert/distilbert-base-uncased-finetuned-sst-2-english',
                              device='cpu')
sentiment_pipeline(reviews)

Die Ergebnisse schwanken in Abhängigkeit des verwendeten Modells. DistilBERT scheint beispielsweise nicht korrekt mit der im dritten Statement enthaltenen, impliziten Aussage umgehen zu können. In der Regel erzeugen neuere Modell oder solche mit mehr Parametern bessere Ergebnisse.

## Keyphrase- und Keyword-Extraktion
Keyphrase-Extraktion ordnet einem Text die in ihm enthaltenen, wichtigsten Terme zu. Verwendet werden die Keyphrases dann zum Beispiel, um eine Zusammenfassung oder eine thematische Beschreibung zu erzeugen. (Die Keyphrase-Extraktion sollte nicht mit Keyword-Assignment verwechselt werden. Letztere extrahiert keine Schlüsselworte, sondern weist eines oder mehrere aus einer vorher bestimmten Menge zu und ist damit eine Klassifikationsaufgabe.)

In einem vorangegangenen Abschnitt haben wir bereits festgestellt, dass TF-IDF den einzelnen Tokens einen Wert zuordnet, welcher die Relevanz widerspiegelt.

In [ ]:
count = CountVectorizer(stop_words=list(stopwords_and_punctuation))
tfidf = TfidfTransformer()

counts = count.fit_transform(reviews)
tfidf = tfidf.fit_transform(counts)
tokens = np.array(count.get_feature_names_out())

tokens

Sortieren wir den TF-IDF Vektor für ein Dokument absteigend, dann erhalten wir damit die relevantesten Tokens für eben dieses Dokument.

In [ ]:
top_n = 2

for i, review in enumerate(reviews):
    review_tfidf = tfidf[i].toarray().flatten()
    sorted_indices = np.argsort(review_tfidf)[::-1]

    print(review)
    print([(tokens[index], review_tfidf[index]) for index in sorted_indices[:top_n]])
    print()

Wie üblich gibt es eine Python-Bibliothek, die komplexere Ansätze implementiert. [pke](https://github.com/boudinfl/pke) verwendet SpaCy und stellt [eine ganze Reihe](https://github.com/boudinfl/pke?tab=readme-ov-file#implemented-models) von Methoden zur Verfügung.

In [ ]:
top_n = 2

extractor = pke.unsupervised.TextRank()

for review in reviews:
    extractor.load_document(input=review, language='en')
    extractor.candidate_selection()
    extractor.candidate_weighting()

    print(review)
    print(extractor.get_n_best(n=top_n))
    print()

## Topic Modeling
Topic Modeling bestimmt Themen in einem Text in Abhängigkeit der Häufung von Wörtern. Dabei wird aber kein abstraktes Oberthema zugeordnet, sondern eine Gruppe von Wörtern aus dem Text als Thema bestimmt.

*Latent Dirichlet Allocation* ist ein Verfahren, das jedes Dokument als eine Ansammlung von Wörtern annimmt. Es wird davon ausgegangen, dass jedes Dokument aus mehreren *verborgenen* Themen (latent topics) besteht, die sich jeweils durch eine Menge von Wörtern beschreiben lassen.

In [ ]:
review_tokens = [
    [token for token in word_tokenize(review) if token not in stopwords_and_punctuation]
    for review in reviews
]

id2word = corpora.Dictionary(review_tokens)
corpus = [id2word.doc2bow(r) for r in review_tokens]

lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=5)

lda_model.print_topics()

Das Ergebnis enthält die vorgegebene Anzahl an Themen, die jeweils aus einer Menge von Termen besteht. Den Thermen wird jeweisl eine Wahrscheinlichkeit zugeordnet, dass sie zu diesem bestimmten Thema gehören.

## Zusammenfassung
In diesem Abschnitt wurden einige praktische Ansätze der Textverarbeitung vorgestellt. Sie sollten diese noch einmal auf Texte mit größerem Umfang anwenden, um aussagekräftigere Ergebnisse zu erhalten.